In [1]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

ISBN = "9789573317241"
title = "哈利波特(1)：神秘的魔法石"

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器

In [ ]:
# 臺北市立圖書館
# 進入《臺北市立圖書館》館藏系統
driver = webdriver.Chrome(options=my_options)
url = "https://book.tpml.edu.tw/webpac/webpacIndex.jsp"
driver.get(url)

# 定位選擇欄，以 ISBN 搜尋方式搜尋
select = Select(driver.find_element_by_name("search_field"))
select.select_by_visible_text(u"ISBN")
search_input = driver.find_element_by_name("search_input")
search_input.send_keys(ISBN)
search_input.submit()

In [ ]:
# 取得當前網頁的 html 進行解析，以取得 DataFrame
html_text = driver.page_source

In [ ]:
dfs = pd.read_html(html_text, encoding="utf-8")
tgt2 = dfs[19]

# 清理資料
tgt2["索書號"] = tgt2["索書號"].fillna(method="ffill", axis=0) + " " + tgt2["編號"].apply(str)
tgt2.drop(["編號", "館藏類型", "註記"], axis=1, inplace=True)
tgt2.rename(columns={
    "典藏館": "館藏地",
    "館藏位置(到期日期僅為期限，不代表上架日期)": "館藏狀態"
}, inplace=True)
tgt2.insert(0, "圖書館", ["臺北市立圖書館" for i in range(tgt2.shape[0])])

In [2]:
# 新北市立圖書館
# 進入《新北市立圖書館》館藏系統
driver = webdriver.Chrome(options=my_options)
url = "https://webpac.tphcc.gov.tw/webpac/search.cfm"
driver.get(url)

# 進階搜尋，以 ISBN 搜尋方式搜尋
driver.find_element_by_xpath('//*[@id="change_adv"]').click()
select = Select(driver.find_element_by_name("as_type"))
select.select_by_visible_text(u"ISBN/ISSN")
search_input = driver.find_element_by_name("as_keyword")
search_input.send_keys(ISBN)
# search_input.submit()  # 不知道為什麼無法 submit()？
search_input.send_keys(Keys.ENTER)  # 改使用鍵盤控制，按下「ENTER」鍵

# 進入詳細書目
time.sleep(1)
driver.find_element_by_xpath('//*[@id="autolink"]/h3').click()

# 選擇書目排序方式：館藏狀態(由小到大)
time.sleep(1)
select = Select(driver.find_element_by_name("sort"))
select.select_by_visible_text(u"館藏狀態(由小到大)")

In [3]:
# 確定搜尋結果的頁數，跑迴圈用
num_of_pages = driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div/div[2]/div[5]/div/div[6]/div/span').text\
.replace("1 - ", "")\
.replace("頁，跳頁至", "")
num_of_pages

'18'

In [4]:
df_lst = []
for pagination in range(1, int(num_of_pages) + 1):
    go_page = driver.find_element_by_name('page')
    go_page.send_keys(pagination)
    go_page.send_keys(Keys.ENTER)
    time.sleep(1)
    
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    df_lst.append(dfs[0])

In [5]:
tgt1 = pd.concat(df_lst, axis=0, ignore_index=True)
tgt1.drop(["條碼號", "館藏流通類別", "資料類型", "預約人數", "備註", "附件", "預約"], axis=1, inplace=True)
tgt1.rename(columns={
    "典藏地名稱": "館藏地"
}, inplace=True)
tgt1.insert(0, "圖書館", ["新北市立圖書館" for i in range(tgt1.shape[0])])

In [7]:
# all-in-one
df_final = pd.concat([tgt1], axis=0, ignore_index=True)
df_final

,圖書館,館藏地,索書號,館藏狀態
0,新北市立圖書館,淡水青少年專區,Y 873.57 857,在架
1,新北市立圖書館,汐止大同分館開架閱覽[原汐止分館],873.57 857 2000,在架
2,新北市立圖書館,汐止大同分館開架閱覽[原汐止分館],873.57 857 2000 c.2,在架
3,新北市立圖書館,新莊開架閱覽,873.57 857 c.2,在架
4,新北市立圖書館,林口開架閱覽,873.57 857 c.2,在架
...,...,...,...,...
168,新北市立圖書館,土城清水開架閱覽,873.57 8666,久借未還 : 2012/11/30
169,新北市立圖書館,三重開架閱覽,873.57 857,D破損
170,新北市立圖書館,三重開架閱覽,873.57 857 c.2,M圖書掛失(開架遺失)
171,新北市立圖書館,三重五常罕用書庫,873.57 857 1櫃,M圖書掛失(開架遺失)


In [ ]:
df_final["館藏地"] = df_final["館藏地"].str.replace("栅", "柵")

In [ ]:
# 輸出成 csv 格式
df_final.to_csv(f"《{title}》的搜尋結果.csv", encoding="big5", index=True, errors="strict")

In [ ]:
# 輸出成 excel
abc = df_final.to_excel(f"《{title}》的搜尋結果.xlsx", encoding="big5", index=True)